In [3]:
import os

import cv2
import numpy as np
import re
import glob
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
from scipy.stats import entropy
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [22]:
import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier

# create an empty dictionary to hold the image data
bills = {'MX020':[],'MX050':[],'MX100':[],'MX200':[],'MX500':[]}
labels = {'MX020':0,'MX050':1,'MX100':2,'MX200':3,'MX500':4}

# define the target size for the images
target_size = (640, 480) #Camera resolution

# load the image data into the dictionary and resize each image
for filename in os.listdir('billetes'):
    for bill in bills:
        if bill in filename:
            img = cv2.imread(os.path.join('billetes',filename))
            if img is not None:
                img_resized = cv2.resize(img, target_size)
                bills[bill].append(img_resized)

# convert the lists to arrays
X = np.concatenate([np.reshape(bills[bill], (len(bills[bill]), -1)) for bill in bills])
# create an array of labels for each image
y = np.concatenate([np.full(len(bills[bill]), labels[bill]) for bill in bills])

# create and train the classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X, y)

# capture an image from the webcam and preprocess it
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
img_resized = cv2.resize(frame, target_size)
gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
gray_flat = np.reshape(gray, (1, -1))   # reshape the grayscale image array to a 1D array


# make a prediction using the trained classifier
pred = clf.predict(gray_flat)
if len(pred) > 0:
    for bill, label in labels.items():
        if label == pred[0]:
            print('Detected bill denomination:', bill)
else:
    print('No bill detected.')


ValueError: X has 307200 features, but RandomForestClassifier is expecting 921600 features as input.

Camera resolution: 640x480


In [23]:
import os
import cv2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# create an empty dictionary to hold the image data
bills = {'MX020':[],'MX050':[],'MX100':[],'MX200':[],'MX500':[]}

# load the image data into the dictionary
for filename in os.listdir('billetes'):
    for bill in bills:
        if bill in filename:
            img = cv2.imread(os.path.join('billetes',filename))
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (224, 224))  # resize the images to 224x224
                bills[bill].append(img)

# convert the data and labels to NumPy arrays
data = []
labels = []
for bill in bills:
    for img in bills[bill]:
        data.append(img)
        labels.append(bill)
data = np.array(data)
labels = np.array(labels)

# flatten the data and reduce the dimensionality using PCA
data = data.reshape(data.shape[0], -1)
pca = PCA(n_components=50, random_state=42)
data_pca = pca.fit_transform(data)

# split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(data_pca, labels, test_size=0.2, random_state=42)

# train a k-nearest neighbors classifier on the training set
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, y_train)

# evaluate the model on the validation set
y_pred = clf.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)
print('Validation accuracy: %.2f%%' % (accuracy * 100))


Validation accuracy: 100.00%


C:\Users\angel\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [26]:

# initialize the webcam
cap = cv2.VideoCapture(0)

while True:
    # capture an image from the webcam
    ret, frame = cap.read()

    # resize the image to 224x224 and flatten it into a 1D array
    img = cv2.resize(frame, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.reshape(1, -1)

    # apply PCA to the flattened image
    img_pca = pca.transform(img)

    # use the k-nearest neighbors classifier to predict the bill denomination
    pred = clf.predict(img_pca)

    # display the predicted denomination on the image
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50, 50)
    fontScale = 1
    color = (0, 0, 255)
    thickness = 2
    cv2.putText(frame, pred[0], org, font, fontScale, color, thickness, cv2.LINE_AA)

    # display the image
    cv2.imshow('webcam', frame)

    # exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


C:\Users\angel\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\angel\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim